In [2]:
from customerManager import CustomerManager
customer_path = "./config/customer_classes.json"
cManager = CustomerManager(file_path = customer_path)

In [36]:
import numpy as np
from scipy.special import erfc

class Learner():
    def __init__(self):
        self.n_arms = 10
        self.bid = np.linspace(1.0, 10.0, self.n_arms)
        self.t = 0 
        
        
        self.rewards_per_arm = x = [[] for i in range(self.n_arms)]
        self.collected_rewards = []
    
    def update_observations(self, pulled_arm, reward):
        self.rewards_per_arm[pulled_arm].append(reward)
        self.collected_rewards.append(reward*10000)

class TS_Learner(Learner):
    def __init__(self, safety_t = 0.2):
        super().__init__()
        self.safety_t = safety_t
        #Mean and Std
        self.normal_parameters = np.zeros((self.n_arms, 2))

    def pull_arm(self):
        if self.t < (2*self.n_arms): return (self.t % self.n_arms) + 1

        #Safety constraint
        #Alternative implementation safety constraints using MATH         
        arg_max = []
        for p in self.normal_parameters : 
            if 0.5 * erfc(p[0]/(np.sqrt(2)*p[1])) > self.safety_t :
                arg_max.append(0.0) 
            else : 
                arg_max.append(np.random.normal(p[0], p[1]))
        

        return np.argmax(arg_max) + 1 #index Bid conversion

    def update(self, pulled_arm, reward):
        self.t += 1
        pulled_arm -= 1  #Bid index conversion
        times_pulled = max(1, len(self.rewards_per_arm[pulled_arm]))
        self.update_observations(pulled_arm, reward)
        #Empirical mean
        self.normal_parameters[pulled_arm, 0] = (self.normal_parameters[pulled_arm, 0] * (times_pulled - 1) + reward ) / times_pulled
        #Empirical std
        #NOt gooooood Should depend on the number of sampl (inversionally proportional)
        self.normal_parameters[pulled_arm, 1] = np.sqrt(sum((self.normal_parameters[pulled_arm, 0] - self.rewards_per_arm[pulled_arm])**2)/times_pulled**2)


In [2]:
opt_bids = [3.8622484787564275 , 2.1216094606111944,  2.347134281066495]
opt_price =6.321089806558111



In [37]:
from environment import Environment
from tqdm import tqdm

price = opt_price

T = 365
env = Environment(noise_variance= 0.05)
regret = []
ts_learner = TS_Learner()
ts_rewards_per_experiment = np.ndarray([])

regret=[]
sols = []

for t in tqdm(range(T)):
    #TS learner 
    bid = ts_learner.pull_arm()
    bids = [bid,bid,bid]
    sols.append(bid)
    reward = env.round(bids,price)
    ts_learner.update(bid, sum(reward)/10000)
    regret.append(sum(env.round(opt_bids, opt_price, noise = False)) - sum(reward))



100%|██████████| 365/365 [00:00<00:00, 1786.85it/s]


In [38]:
import matplotlib.pyplot as plt 

plt.figure(0)
plt.xlabel("t")
plt.ylabel("Regret")
plt.plot(regret , 'r')
plt.legend(["TS" ])
plt.show()

In [39]:
regret[-10:]

[100.25695609634931,
 115.65679879502159,
 -166.54619718573485,
 91.9438212023233,
 227.0623176170011,
 -133.22415513440956,
 -73.87605143168958,
 146.92553043609905,
 -122.09142984346454,
 -49.709869457680725]

In [42]:
sols[-10:]

[3, 3, 2, 5, 3, 3, 5, 2, 3, 3]